In [1]:
from __future__ import print_function

from syntax import Identifier, LambdaAbs, Application, LetBinding
from typeDefn import TypeVariable, TypeOperator
from exceptions import InferenceError, ParseError
from inference import *
from typingEnv import typingEnv

In [2]:
def exp(expression):
    print(str(expression) + " : ", end=" ")
    try:
        t = infer(expression, typingEnv)
        print(str(t))
    except (ParseError, InferenceError) as e:
        print(e)

In [4]:
# factorial
factorial = LetBinding(
    "factorial",  # let factorial =
    Application(
        Identifier("fix"),
        LambdaAbs(
            "self",
            LambdaAbs(
                "n",  # fn n =>
                Application(
                    Application(  # cond (zero n) 1
                        Application(
                            Identifier("cond"),  # cond (zero n)
                            Application(Identifier("zero"), Identifier("n")),
                        ),
                        Identifier("1"),
                    ),  # else
                    Application(  # times n
                        Application(Identifier("times"), Identifier("n")),
                        Application(
                            Identifier("self"),
                            Application(
                                Identifier("pred"), Identifier("n")
                            ),  # n - 1
                        ),
                    ),
                ),
            ),
        ),
    ),  # in factorical 5
    Application(Identifier("factorial"), Identifier("5")),
)

exp(factorial)

(let factorial = (fix (fn self => (fn n => (((cond (zero n)) 1) ((times n) (self (pred n))))))) in (factorial 5)) :  int


In [5]:
# let f = (fn x => x) in ((pair (f 4)) (f true))
let1 = LetBinding(
            "f",
            LambdaAbs("x", Identifier("x")),
            Application(
                Application(
                    Identifier("pair"), Application(Identifier("f"), Identifier("4"))
                ),
                Application(Identifier("f"), Identifier("true")),
            ),
        )
exp(let1)

(let f = (fn x => x) in ((pair (f 4)) (f true))) :  (int * bool)


In [6]:
# let g = fn f => 5 in g g
let2 = LetBinding(
    "g",
    LambdaAbs("f", Identifier("5")),
    Application(Identifier("g"), Identifier("g")),
)
exp(let2)

(let g = (fn f => 5) in (g g)) :  int


In [3]:
# fn g => let f = fn x => g in pair (f 3, f true)
non_gen1 = LambdaAbs(
    "g",
    LetBinding(
        "f",
        LambdaAbs("x", Identifier("g")),
        Application(
            Application(
                Identifier("pair"),
                Application(Identifier("f"), Identifier("3")),
            ),
            Application(Identifier("f"), Identifier("true")),
        ),
    ),
)
exp(non_gen1)

(fn g => (let f = (fn x => g) in ((pair (f 3)) (f true)))) :  (a -> (a * a))


In [3]:
# fn f (fn g (fn arg (f g arg)))
non_gen2 = LambdaAbs(
    "f",
    LambdaAbs(
        "g",
        LambdaAbs(
            "arg",
            Application(
                Identifier("g"), Application(Identifier("f"), Identifier("arg"))
            ),
        ),
    ),
)
exp(non_gen2)

(fn f => (fn g => (fn arg => (g (f arg))))) :  ((c -> d) -> ((d -> e) -> (c -> e)))


In [4]:
# fn x => (pair(x(3) (x(true)))
error1 = LambdaAbs(
    "x",
    Application(
        Application(
            Identifier("pair"), Application(Identifier("x"), Identifier("3"))
        ),
        Application(Identifier("x"), Identifier("true")),
    ),
)
exp(error1)

(fn x => ((pair (x 3)) (x true))) :  Type mismatch: bool != int


In [5]:
# pair(f(3), f(true))
error2 = Application(
    Application(
        Identifier("pair"), Application(Identifier("f"), Identifier("4"))
    ),
    Application(Identifier("f"), Identifier("true")),
)
exp(error2)

((pair (f 4)) (f true)) :  Undefined symbol f
